In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt

import utils

import importlib as im
im.reload(utils)

<module 'utils' from 'd:\\AIU\\git\\routes\\utils.py'>

In [2]:
class Hungarian:
    rounding_value = 2

    def __init__(
        self,
        random_generate : bool = False,
        checkpoints_cnt : int = 10,
        minmax : tuple = (5, 15),
        data : list = [[],[]]):
        """инициализация объекта венгерского алгоритма

        Args:
            random_generate (bool, optional): генерация случайной карты. Defaults to False.
            checkpoints_cnt (int, optional): количество точек (используется только при генерации карты). Defaults to 10.
            minmax (tuple, optional): минимальное и максимальное значение времени между двумя точками (используется только при генерации карты). Defaults to (5, 15).
            data (list, optional): карта, заданная пользователем (используется при значении random_generate=False). Defaults to [[],[]].
        """
        if random_generate:
            self.start_field = np.random.randint(minmax[0], minmax[1], size=(checkpoints_cnt, checkpoints_cnt))
        else:
            self.start_field = np.array(data)

        self.max_time = self.start_field.max()+1
        np.fill_diagonal(self.start_field, self.max_time)
        self.checkpoints_cnt = self.start_field.shape[0]
        self.current_field = utils.rounding(
            field=self.start_field.copy(),
            rounding_value=self.__class__.rounding_value)


    def reduce(self) -> None:         
        """Выполнение операции вычитания минимальных значений по строкам и столбцам
        """
        self.current_field = (self.current_field.T - utils.get_min(self.current_field, True)).T
        self.current_field -= utils.get_min(self.current_field, False)
        np.fill_diagonal(self.current_field, self.max_time)


    def check(self) -> np.array:
        """Поиск возможных решений

        Returns:
            np.array: найденные решения
        """
        def recursive_search(checkpoint:int, path:list) -> None:
            """Рекурсивная проверка существования решения

            Args:
                checkpoint (int): номер точки
                path (list): маршрут
            """
            if len(path)==self.checkpoints_cnt:
                exists_path.append(path)
                return
            if checkpoint in path:
                return        
            path.append(checkpoint)        
            for value in checkpoit_times[checkpoint]:
                recursive_search(value, path.copy())                
            
        exists_path = []
        rows, columns = np.where(self.current_field==0)
        checkpoit_times = {i:[] for i in range(self.checkpoints_cnt)}
        for i, rr in enumerate(rows):
            checkpoit_times[rr].append(columns[i])
        for i in range(self.checkpoints_cnt):
            recursive_search(i, [])
        if len(exists_path)==0:
            return exists_path
        return np.unique(exists_path, axis=0)

In [95]:
hung = Hungarian(random_generate=True, checkpoints_cnt=10, minmax=(4, 27))
hung.reduce()
print(hung.current_field)
hung.check()


[[27  2  0  8 16 10  8 12  8 20]
 [10 27 18  2  8 12 20  0  4  0]
 [ 0  4 27 12 14 18  6 10  4 14]
 [12  4 14 27 20  8 18 14  0  6]
 [ 2 16  2 20 27 18  0  8  8 18]
 [ 8 10  2  8  4 27 14 18  6  0]
 [ 2 10 18  2  0  6 27 14 14  8]
 [12 18 18  0  2 20  0 27 20  8]
 [ 4 22 16 14  0 22 14 12 27 16]
 [16  0 14 14 16  0 14  4  4 27]]


[]

In [138]:
class GeneticRemovingZeros:
    population_size = 250
    nsurv = 50
    nnew = population_size - nsurv
    epochs = 500
    mut = 0.25

    def __init__(self, checkpoint_count:int) -> None:
        """Инициализация объекта генетического алгоритма для удаления нулей из временного поля

        Args:
            checkpoint_count (int): Количество контрольных точек
        """
        self.bot_length = checkpoint_count*2
        self.popul = np.random.choice(2, size=(self.__class__.population_size, self.bot_length))


    def get_surv_popul(self, val:np.array) -> np.array:
        """Сортировка популяции в соответствии со значениями оценочной функции

        Args:
            val (np.array): Значения оценочной функции для каждого бота

        Returns:
            np.array: Отсортированная популяция (по убыванию)
        """
        return self.popul[val.argsort()[::-1]]

  
    def cross_point(self, curr_popul: np.array)->np.array:
        """Скрещивание двух родителей

        Args:
            curr_popul (np.array): упорядоченная популяция

        Returns:
            np.array: потомок
        """
        bots = curr_popul[np.random.randint(0, self.__class__.nsurv - 1, size=2)]
        return bots

    
    def process(self, field:np.array)->np.array:
        """Запуск генетического алгоритма

        Args:
            field (np.array): временная карта

        Returns:
            np.array: массив с исключающими колонками и столбцами
        """
        for _ in range(self.__class__.epochs):
            validation = np.zeros(shape=self.__class__.population_size)
            for i, bot in enumerate(self.popul):
                cut_field = utils.cut_matrix(field, bot)
                validation[i] = cut_field.size * (np.where(cut_field==0)[0].size - 1)                 
            print(validation)

In [140]:
genetic = GeneticRemovingZeros(checkpoint_count=3)

In [142]:
genetic.process(hung.current_field)

AttributeError: module 'utils' has no attribute 'cut_matrix'